<a href="https://colab.research.google.com/github/JSJeong-me/AWS_Reinforcement_Learning/blob/main/cart-pole-230101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


n_episode :20, score : 10.3, n_buffer : 207, eps : 7.9%
n_episode :40, score : 9.5, n_buffer : 397, eps : 7.8%
n_episode :60, score : 9.6, n_buffer : 588, eps : 7.7%
n_episode :80, score : 9.6, n_buffer : 779, eps : 7.6%
n_episode :100, score : 9.6, n_buffer : 971, eps : 7.5%
n_episode :120, score : 9.7, n_buffer : 1165, eps : 7.4%
n_episode :140, score : 9.5, n_buffer : 1355, eps : 7.3%
n_episode :160, score : 9.4, n_buffer : 1543, eps : 7.2%
n_episode :180, score : 10.2, n_buffer : 1747, eps : 7.1%
n_episode :200, score : 9.2, n_buffer : 1930, eps : 7.0%


<ipython-input-1-871dc58d492f>:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \


n_episode :220, score : 10.9, n_buffer : 2148, eps : 6.9%
n_episode :240, score : 53.6, n_buffer : 3220, eps : 6.8%
n_episode :260, score : 21.4, n_buffer : 3649, eps : 6.7%
n_episode :280, score : 20.6, n_buffer : 4061, eps : 6.6%
n_episode :300, score : 15.0, n_buffer : 4361, eps : 6.5%
n_episode :320, score : 49.6, n_buffer : 5353, eps : 6.4%
n_episode :340, score : 146.8, n_buffer : 8288, eps : 6.3%
n_episode :360, score : 178.1, n_buffer : 11849, eps : 6.2%
n_episode :380, score : 168.1, n_buffer : 15211, eps : 6.1%
n_episode :400, score : 153.3, n_buffer : 18278, eps : 6.0%
n_episode :420, score : 167.7, n_buffer : 21632, eps : 5.9%
n_episode :440, score : 134.8, n_buffer : 24327, eps : 5.8%
n_episode :460, score : 168.2, n_buffer : 27690, eps : 5.7%
n_episode :480, score : 266.5, n_buffer : 33020, eps : 5.6%
n_episode :500, score : 272.9, n_buffer : 38477, eps : 5.5%
n_episode :520, score : 245.9, n_buffer : 43396, eps : 5.4%
n_episode :540, score : 273.8, n_buffer : 48871, eps 